# 🌍 DRRA-Agent  
### AI-Powered Disaster Relief & Resource Allocation Agent

This is my capstone project for the **Kaggle 5-Day AI Agents Intensive**.

I designed and implemented a **multi-agent system** that helps with **disaster relief and resource allocation**. The goal is to go from a natural language description of a disaster (like “severe flood in coastal city of X”) to:

- a structured **relief plan**, and  
- a practical **action checklist** for responders.

In this notebook I show:

- 🧠 How I built my **multi-agent pipeline**  
- 🔧 How I used a **custom tool** and scoring logic  
- 🧾 How I added **memory** to store previous disasters  
- 📊 How I used **logging** for basic observability  
- 🤖 How I integrated **Gemini (gemini-1.5-flash)** using Kaggle Secrets  


## 1. Install Dependencies
I keep dependencies minimal so the notebook runs easily on Kaggle.



In [ ]:
!pip install google-generativeai python-dateutil

: 

## 2. Imports
Here I import everything I need for the agents, logging, time and Gemini.


In [ ]:
import os
import json
import random
from datetime import datetime
import google.generativeai as genai
from dateutil import tz


## 3. Configure Gemini (using Kaggle Secrets)

I don’t hardcode my API key.  
Instead, I use **Kaggle Secrets**:

1. I went to: *Notebook* → **Settings** → **Secrets**  
2. I created a secret named **`GEMINI_API_KEY`**  
3. I paste my Gemini key there

Here I load that key safely and configure the client.


In [ ]:
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
api_key = user_secrets.get_secret("GEMINI_API_KEY")

if api_key:
    genai.configure(api_key=api_key)
    print("✔ Gemini API Key loaded safely using Kaggle Secrets")
else:
    print("❌ GEMINI_API_KEY not found. Please add it in Kaggle Secrets.")


## 4. Simple Logger (Observability)

To keep things transparent, I use a very small logger that prints which agent is doing what. This gives me a quick trace of the full pipeline.



In [ ]:
class SimpleLogger:
    def log(self, agent, message):
        print(f"[{agent}] {message}")


## 5. Base Agent Class

All my agents inherit from this base class so they share a common structure (name, description, logger).


In [ ]:
class BaseAgent:
    def __init__(self, name, description, logger):
        self.name = name
        self.description = description
        self.logger = logger

    def run(self, *args, **kwargs):
        raise NotImplementedError("Agent must implement run()")


## 6. Data Collector Agent

This is the first agent in my pipeline.  
In a real system it could call external tools/APIs.  
Here I simulate realistic metrics so that the rest of the agents can still behave meaningfully.



In [ ]:
class DataCollectorAgent(BaseAgent):
    def run(self, query):
        self.logger.log(self.name, f"Collecting data for query: '{query}'")

        raw_reports = [
            "Heavy impact reported in low-lying areas.",
            "Multiple roads blocked due to debris or flooding.",
            "Reports of power cuts and disruption of basic services.",
        ]

        metrics = {
            "query": query,
            "location": query,
            "disaster_type": "flood" if "flood" in query.lower() else "unknown",
            "people_affected": random.randint(50, 200),
            "severity_index": random.randint(5, 10),
            "infra_damage_index": random.randint(3, 8),
            "raw_reports": raw_reports,
        }

        self.logger.log(self.name, f"Collected metrics: {metrics}")
        return metrics


## 7. Resource Analyzer Agent

This agent takes the collected data and turns it into a structured **analysis**:

- a simple **risk score**  
- inferred **priority needs**  
- cleaned metadata about the disaster


In [ ]:
class ResourceAnalyzerAgent(BaseAgent):
    def run(self, data):
        self.logger.log(self.name, "Analyzing collected data...")

        risk = (
            data["severity_index"] * 6 +
            data["infra_damage_index"] * 3 +
            data["people_affected"] * 0.5
        )

        analysis = {
            "location": data["location"],
            "disaster_type": data["disaster_type"],
            "risk_score": round(risk, 1),
            "priority_needs": [
                "Clean drinking water", "Emergency rescue teams",
                "Food supplies", "Medical camps"
            ],
            "people_affected": data["people_affected"],
            "severity_index": data["severity_index"],
            "infra_damage_index": data["infra_damage_index"],
        }

        self.logger.log(self.name, f"Analysis result: {analysis}")
        return analysis


## 8. Strategy Planner Agent (Gemini-powered)

This is the main AI part of my system.

Here I use **Gemini (gemini-1.5-flash)** to turn the structured analysis into a full disaster relief plan. If Gemini is unavailable for any reason, I fall back to a deterministic template so the pipeline still works.



In [ ]:
class StrategyPlannerAgent(BaseAgent):
    def __init__(self, name, description, logger, use_gemini=True):
        super().__init__(name, description, logger)
        self.use_gemini = use_gemini

    def run(self, analysis):
        self.logger.log(self.name, "Planning disaster relief strategy...")

        if self.use_gemini and api_key:
            try:
                model = genai.GenerativeModel("gemini-1.5-flash")
                prompt = f"""
                You are a professional disaster-response planner.
                Create a structured disaster relief strategy with the following sections:

                1. Situation Summary  
                2. Priority Needs  
                3. Immediate Actions (0–24 hours)  
                4. Short-Term Actions (1–3 days)  
                5. Medium-Term Actions (3–7 days)  
                6. Risks & Assumptions  

                Disaster data:
                {analysis}
                """
                response = model.generate_content(prompt)
                self.logger.log(self.name, "✔ Plan generated using Gemini")
                return response.text
            except Exception as e:
                self.logger.log(self.name, f"Gemini failed → fallback mode ({e})")

        fallback = f"""
Fallback Relief Plan for {analysis['location']}

Immediate Steps:
- Provide water and food  
- Deploy rescue teams  
- Medical tents and emergency kits  
- Setup temporary shelters

(This is a non-AI fallback in case Gemini is unavailable.)
"""
        return fallback


## 9. Deployment Agent

This agent converts the textual plan into a **clear checklist** that could be used by field teams during execution.


In [ ]:
class DeploymentAgent(BaseAgent):
    def run(self, plan_text):
        self.logger.log(self.name, "Converting plan into checklist...")

        checklist = [
            "Deploy assessment teams to affected areas.",
            "Set up medical and first-aid stations.",
            "Distribute clean water and food.",
            "Arrange temporary shelters.",
            "Clear blocked roads for rescue vehicles.",
            "Set up communication control room.",
            "Begin infrastructure repair prioritization."
        ]

        self.logger.log(self.name, f"Generated checklist with {len(checklist)} items.")
        return checklist


## 10. Memory Bank

I also wanted my agent system to “remember” past disasters.

So I created a very simple in-notebook **MemoryBank** that stores recent disaster contexts, so I can later extend it to patterns like “similar past floods”.


In [ ]:
class MemoryBank:
    def __init__(self):
        self.memory = []

    def add(self, item):
        self.memory.append(item)

    def last(self, n=3):
        return self.memory[-n:]


## 11. Disaster Context Builder

After each run, I store a compact summary of the disaster in memory. This function builds that summary dictionary.


In [ ]:
def build_disaster_context(analysis, plan):
    return {
        "location": analysis["location"],
        "severity_level": "high" if analysis["risk_score"] > 70 else "medium",
        "summary": f"Risk {analysis['risk_score']}, needs: {analysis['priority_needs']}",
        "timestamp": datetime.now().isoformat(),
    }


## 12. DRRA-Agent Orchestrator

Here I connect all my agents together into one **end-to-end system**:

1. DataCollectorAgent  
2. ResourceAnalyzerAgent  
3. StrategyPlannerAgent (Gemini)  
4. DeploymentAgent  
5. MemoryBank  

This orchestrator represents my final multi-agent disaster relief pipeline.


In [ ]:
class DRRAAgentSystem:
    def __init__(self, logger, use_gemini=True):
        self.logger = logger
        self.memory = MemoryBank()

        self.collector = DataCollectorAgent("DataCollectorAgent", "", logger)
        self.analyzer = ResourceAnalyzerAgent("ResourceAnalyzerAgent", "", logger)
        self.planner = StrategyPlannerAgent("StrategyPlannerAgent", "", logger, use_gemini)
        self.deployer = DeploymentAgent("DeploymentAgent", "", logger)

    def run(self, query):
        self.logger.log("DRRAAgentSystem", "=== Starting full pipeline ===")

        data = self.collector.run(query)
        analysis = self.analyzer.run(data)
        plan = self.planner.run(analysis)
        checklist = self.deployer.run(plan)

        context = build_disaster_context(analysis, plan)
        self.memory.add(context)

        return {
            "plan": plan,
            "checklist": checklist,
            "memory_snapshot": self.memory.last()
        }


## 13. Demo Run

Now I run the full DRRA-Agent system on a sample query.  
I can change the query text to test different disaster scenarios.


In [ ]:
logger = SimpleLogger()
system = DRRAAgentSystem(logger=logger, use_gemini=True)

query = "Severe flood in coastal city of X"
result = system.run(query)

print("\n===== RELIEF PLAN =====\n")
print(result["plan"])

print("\n===== CHECKLIST =====\n")
for i, item in enumerate(result["checklist"], start=1):
    print(f"{i}. {item}")

print("\n===== MEMORY SNAPSHOT =====\n")
print(json.dumps(result["memory_snapshot"], indent=2))


## 14. What I Achieved With This Project 🎯

In this capstone project I:

- Designed and implemented my own **multi-agent architecture** for disaster relief  
- Used a **custom scoring & analysis step** to estimate risk and needs  
- Integrated **Gemini** to generate professional-style relief plans  
- Added a **fallback planner** so the system is robust  
- Implemented a small **memory bank** to store recent disaster contexts  
- Added **logging** so I can trace each step of the pipeline

This notebook is the main artifact of my submission and the code behind my GitHub repository for the DRRA-Agent project.

